# Step 1: Download GPT-2 Model Weights from HuggingFace

In this notebook, we'll download GPT-2 in **SafeTensors format** and understand what's inside the model weights.

## What We'll Learn

1. **What are Tensors?** - Multi-dimensional arrays that store model parameters
2. **Token Embeddings** - How words become numbers
3. **Position Embeddings** - How the model knows word order
4. **Attention Heads** - How the model focuses on different parts of input
5. **SafeTensors Format** - A simple, transparent file format for model weights

---

## What is a Tensor?

A **tensor** is essentially a multi-dimensional array of numbers. In machine learning:

| Dimensions | Name | Example in GPT-2 |
|------------|------|------------------|
| 0D | Scalar | A single number |
| 1D | Vector | Layer norm bias `[768]` |
| 2D | Matrix | Token embeddings `[50257, 768]` |
| 3D+ | Tensor | Batched inputs, attention weights, etc. |

The model's **learned weights** are stored as tensors - millions of floating-point numbers that were optimized during training.


## Setup and Imports


In [20]:
import os
import json
import requests
import numpy as np
from tqdm.notebook import tqdm  # Use notebook version for better display


## Configuration

We'll download GPT-2 (124M parameters) from HuggingFace. The model consists of:
- **model.safetensors**: The actual model weights (~548 MB)
- **config.json**: Architecture hyperparameters (layers, heads, dimensions)
- **vocab.json**: Vocabulary mapping (token string → token ID)
- **merges.txt**: BPE merge rules for tokenization


In [21]:
# HuggingFace model repository
MODEL_REPO = "openai-community/gpt2"
HF_BASE_URL = f"https://huggingface.co/{MODEL_REPO}/resolve/main"

# Files we need
FILES_TO_DOWNLOAD = {
    "model.safetensors": "The model weights (548 MB)",
    "config.json": "Model architecture config",
    "vocab.json": "Vocabulary: token string -> token ID",
    "merges.txt": "BPE merge rules for tokenization",
}

MODELS_DIR = "models/gpt2"


## Download Functions


In [22]:
def download_file(url: str, dest: str, desc: str = None):
    """Download a file with progress bar."""
    response = requests.get(url, stream=True, allow_redirects=True)
    response.raise_for_status()
    
    total_size = int(response.headers.get('content-length', 0))
    desc = desc or os.path.basename(dest)
    
    with open(dest, 'wb') as f:
        with tqdm(total=total_size, unit='B', unit_scale=True, desc=desc) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                pbar.update(len(chunk))


def download_gpt2(models_dir: str = MODELS_DIR) -> str:
    """Download GPT-2 files from HuggingFace."""
    os.makedirs(models_dir, exist_ok=True)
    
    print(f"📥 Downloading GPT-2 from HuggingFace ({MODEL_REPO})\n")
    
    for filename, description in FILES_TO_DOWNLOAD.items():
        dest = os.path.join(models_dir, filename)
        
        if os.path.exists(dest):
            print(f"✓ {filename} already exists")
            continue
        
        url = f"{HF_BASE_URL}/{filename}"
        print(f"Downloading {filename} - {description}")
        download_file(url, dest, filename)
        print()
    
    print(f"✅ All files downloaded to {models_dir}/")
    return models_dir


## Understanding SafeTensors Format

SafeTensors is a simple, transparent file format for storing tensors. Unlike pickle-based formats, it's:
- **Safe**: No arbitrary code execution
- **Fast**: Memory-mapped loading
- **Simple**: Easy to parse without external libraries!

### File Structure

```
┌──────────────────────────────────────────────────────────────┐
│ Bytes 0-7: Header size (uint64, little-endian)               │
├──────────────────────────────────────────────────────────────┤
│ Bytes 8 to 8+N: JSON header with tensor metadata             │
│   {                                                          │
│     "tensor_name": {                                         │
│       "dtype": "F32",                                        │
│       "shape": [768, 768],                                   │
│       "data_offsets": [start, end]                           │
│     },                                                       │
│     ...                                                      │
│   }                                                          │
├──────────────────────────────────────────────────────────────┤
│ Rest: Raw tensor data (binary)                               │
└──────────────────────────────────────────────────────────────┘
```


In [23]:
def load_safetensors(path: str) -> dict:
    """
    Load a SafeTensors file WITHOUT the safetensors library!
    
    SafeTensors format is beautifully simple:
    1. First 8 bytes: header size (uint64, little-endian)
    2. Next N bytes: JSON header with tensor metadata
    3. Rest: raw tensor data
    
    This is why SafeTensors is great for learning - it's transparent!
    """
    print(f"\n📦 Loading SafeTensors from {path}")
    print("   (Parsing the raw binary format - no library needed!)\n")
    
    with open(path, 'rb') as f:
        # Step 1: Read header size (first 8 bytes)
        header_size = int.from_bytes(f.read(8), 'little')
        print(f"   Header size: {header_size} bytes")
        
        # Step 2: Read and parse JSON header
        header_json = f.read(header_size).decode('utf-8')
        header = json.loads(header_json)
        
        # The header contains metadata and tensor info
        # "__metadata__" key contains file-level metadata (optional)
        metadata = header.pop("__metadata__", {})
        
        print(f"   Number of tensors: {len(header)}")
        print(f"   Metadata: {metadata}\n")
        
        # Step 3: Load each tensor
        tensors = {}
        data_start = 8 + header_size  # Where tensor data begins
        
        for name, info in header.items():
            dtype_str = info["dtype"]
            shape = info["shape"]
            offset_start, offset_end = info["data_offsets"]
            
            # Map SafeTensors dtype to numpy dtype
            dtype_map = {
                "F32": np.float32,
                "F16": np.float16,
                "BF16": np.float16,  # We'll handle bfloat16 specially if needed
                "I64": np.int64,
                "I32": np.int32,
            }
            dtype = dtype_map.get(dtype_str, np.float32)
            
            # Read the raw bytes
            f.seek(data_start + offset_start)
            num_bytes = offset_end - offset_start
            raw_data = f.read(num_bytes)
            
            # Convert to numpy array
            tensor = np.frombuffer(raw_data, dtype=dtype).reshape(shape)
            tensors[name] = tensor
        
        return tensors


## Download and Load the Model

Let's download GPT-2 and load the weights!


In [24]:
# Download the model
model_dir = download_gpt2()

# Load the weights (parsing SafeTensors ourselves!)
safetensors_path = os.path.join(model_dir, "model.safetensors")
tensors = load_safetensors(safetensors_path)

# Load the config
config_path = os.path.join(model_dir, "config.json")
with open(config_path) as f:
    config = json.load(f)

print(f"\nLoaded {len(tensors)} tensors from the model!")


📥 Downloading GPT-2 from HuggingFace (openai-community/gpt2)

✓ model.safetensors already exists
✓ config.json already exists
✓ vocab.json already exists
✓ merges.txt already exists
✅ All files downloaded to models/gpt2/

📦 Loading SafeTensors from models/gpt2/model.safetensors
   (Parsing the raw binary format - no library needed!)

   Header size: 14283 bytes
   Number of tensors: 160
   Metadata: {'format': 'pt'}




Loaded 160 tensors from the model!


---

## Understanding Token Embeddings (`wte.weight`)

**Token embeddings** convert each token (word/subword) into a vector of numbers.

In GPT-2, this is a **lookup table** with shape `(50257, 768)`:
- **50257** = vocabulary size (all possible tokens)
- **768** = embedding dimension

### How it works:

```
"Hello" → Token ID 15496 → Look up row 15496 in wte.weight → [0.12, -0.34, 0.56, ... 768 numbers]
```

Each token gets a unique 768-dimensional vector that captures its **semantic meaning**. Words with similar meanings end up with similar vectors.


In [25]:
# Let's examine the token embeddings
wte = tensors["wte.weight"]
print(f"Token Embeddings (wte.weight)")
print(f"  Shape: {wte.shape}")
print(f"  - {wte.shape[0]:,} tokens in vocabulary")
print(f"  - {wte.shape[1]} dimensions per token")
print(f"  Total parameters: {wte.size:,}")
print(f"\nExample: First 10 values of token ID 0:")
print(f"  {wte[0, :10]}")


Token Embeddings (wte.weight)
  Shape: (50257, 768)
  - 50,257 tokens in vocabulary
  - 768 dimensions per token
  Total parameters: 38,597,376

Example: First 10 values of token ID 0:
  [-0.11010301 -0.03926672  0.03310751  0.13382645 -0.04847569 -0.07891767
 -0.23977417 -0.08947388  0.02525497 -0.10739683]


## Understanding Position Embeddings (`wpe.weight`)

**Position embeddings** tell the model *where* each token appears in the sequence.

In GPT-2, shape `(1024, 768)`:
- **1024** = max sequence length (context window)
- **768** = embedding dimension

### Why position embeddings are needed:

Transformers process all tokens in **parallel** (unlike RNNs which go step-by-step). Without position info:

```
"The cat sat on the mat"  ←→  "mat the on sat cat The"
```

These would look identical to the model! Position embeddings let the model know token order.

### How Token + Position Embeddings Combine:

```
Input to GPT-2 = Token Embedding + Position Embedding
```

| Position | Token | Token Embedding | Position Embedding | Combined Input |
|----------|-------|-----------------|-------------------|----------------|
| 0 | "Hello" | `wte[15496]` | `wpe[0]` | `wte[15496] + wpe[0]` |
| 1 | "world" | `wte[995]` | `wpe[1]` | `wte[995] + wpe[1]` |

This gives each token a unique representation that encodes **both** its meaning **and** its position.


In [26]:
# Let's examine the position embeddings
wpe = tensors["wpe.weight"]
print(f"Position Embeddings (wpe.weight)")
print(f"  Shape: {wpe.shape}")
print(f"  - {wpe.shape[0]:,} max positions (context window)")
print(f"  - {wpe.shape[1]} dimensions per position")
print(f"  Total parameters: {wpe.size:,}")
print(f"\nExample: First 10 values of position 0:")
print(f"  {wpe[0, :10]}")


Position Embeddings (wpe.weight)
  Shape: (1024, 768)
  - 1,024 max positions (context window)
  - 768 dimensions per position
  Total parameters: 786,432

Example: First 10 values of position 0:
  [-0.01882072 -0.1974186   0.00402672  0.01134686  0.06382412 -0.10501328
  0.03693705 -0.16802956 -0.04911101 -0.05646128]


---

## Understanding Attention Heads

An **attention head** is a mechanism that learns to **focus on different parts of the input** when processing each token. Think of it as one "perspective" the model uses to understand relationships between words.

### Why Multiple Heads?

GPT-2 uses **12 attention heads** (in the 124M model). Each head can learn to pay attention to **different patterns**:

| Head | What it might learn |
|------|---------------------|
| Head 1 | Subject-verb relationships ("The **cat** *sits*") |
| Head 2 | Adjective-noun pairs ("**red** *apple*") |
| Head 3 | Pronouns to their referents ("John... **he**") |
| Head 4 | Next word patterns |
| ... | Different linguistic patterns |

### How They Work Together

```
Input embedding (768 dimensions)
         │
         ▼
    ┌────┴────┐
    │  Split  │  768 ÷ 12 = 64 dims per head
    └────┬────┘
         │
    ┌────┼────┬────┬────┬─── ... ───┐
    ▼    ▼    ▼    ▼    ▼           ▼
  Head  Head  Head  Head  ...     Head
   1     2     3     4             12
  (64)  (64)  (64)  (64)          (64)
    │    │    │    │               │
    └────┴────┴────┴─── ... ───────┘
         │
         ▼
    ┌────┴────┐
    │ Concat  │  → Back to 768 dimensions
    └─────────┘
```

### The Math

```python
n_embd = 768      # Total embedding dimension
n_head = 12       # Number of attention heads
head_dim = n_embd // n_head  # = 64 dimensions per head
```

Each head works with a **64-dimensional slice** of the full 768-dimensional space. The outputs from all 12 heads are concatenated back together.

### Why This Design?

**Multi-head attention** allows the model to:
1. Learn **diverse patterns** (each head specializes differently)
2. Process in **parallel** (all heads compute simultaneously)
3. Capture **both local and global** relationships

It's like having 12 different experts look at the same sentence, each focusing on different linguistic aspects, then combining their insights!


In [27]:
# Get attention head configuration from config.json
n_head = config["n_head"]
n_embd = config["n_embd"]
n_layer = config["n_layer"]
head_dim = n_embd // n_head

print("Attention Head Configuration (from config.json)")
print(f"  n_head (number of heads):     {n_head}")
print(f"  n_embd (embedding dimension): {n_embd}")
print(f"  head_dim (per-head dim):      {head_dim} (= {n_embd} ÷ {n_head})")
print(f"  n_layer (transformer blocks): {n_layer}")

# Show the attention weight that projects to Q, K, V
c_attn = tensors["h.0.attn.c_attn.weight"]
print(f"\nAttention QKV projection weight (h.0.attn.c_attn.weight)")
print(f"  Shape: {c_attn.shape}")
print(f"  - Projects {c_attn.shape[0]} → {c_attn.shape[1]} (= 3 × {n_embd} for Q, K, V)")


Attention Head Configuration (from config.json)
  n_head (number of heads):     12
  n_embd (embedding dimension): 768
  head_dim (per-head dim):      64 (= 768 ÷ 12)
  n_layer (transformer blocks): 12

Attention QKV projection weight (h.0.attn.c_attn.weight)
  Shape: (768, 2304)
  - Projects 768 → 2304 (= 3 × 768 for Q, K, V)


---

## Exploring All Model Weights

Let's look at all the tensors in GPT-2 and understand what each one does.


In [28]:
def explore_weights(tensors: dict):
    """Print information about all tensors in the model."""
    
    print("=" * 75)
    print("🔍 GPT-2 Weight Tensors")
    print("=" * 75)
    
    total_params = 0
    
    # Group by component
    embeddings = {}
    blocks = {}
    final_ln = {}
    
    for name, tensor in sorted(tensors.items()):
        params = tensor.size
        total_params += params
        
        if name.startswith("wte") or name.startswith("wpe"):
            embeddings[name] = tensor
        elif name.startswith("h."):
            # Extract block number
            parts = name.split(".")
            block_num = int(parts[1])
            if block_num not in blocks:
                blocks[block_num] = {}
            blocks[block_num][name] = tensor
        else:
            final_ln[name] = tensor
    
    # Print embeddings
    print("\n📊 EMBEDDINGS")
    print("-" * 75)
    for name, tensor in embeddings.items():
        print(f"  {name:40} {str(tensor.shape):20} {tensor.size:>12,} params")
    
    # Print one block in detail
    print("\n📊 TRANSFORMER BLOCKS (showing block 0 as example)")
    print("-" * 75)
    if 0 in blocks:
        for name, tensor in sorted(blocks[0].items()):
            short_name = name.replace("h.0.", "")
            print(f"  {short_name:40} {str(tensor.shape):20} {tensor.size:>12,} params")
    
    print(f"\n  ... (blocks 1-11 have the same structure)")
    
    # Print final layer norm
    print("\n📊 FINAL LAYER NORM")
    print("-" * 75)
    for name, tensor in final_ln.items():
        print(f"  {name:40} {str(tensor.shape):20} {tensor.size:>12,} params")
    
    print("\n" + "=" * 75)
    print(f"📈 TOTAL PARAMETERS: {total_params:,}")
    print("=" * 75)
    
    return total_params

# Explore the weights
total_params = explore_weights(tensors)


🔍 GPT-2 Weight Tensors

📊 EMBEDDINGS
---------------------------------------------------------------------------
  wpe.weight                               (1024, 768)               786,432 params
  wte.weight                               (50257, 768)           38,597,376 params

📊 TRANSFORMER BLOCKS (showing block 0 as example)
---------------------------------------------------------------------------
  attn.bias                                (1, 1, 1024, 1024)      1,048,576 params
  attn.c_attn.bias                         (2304,)                     2,304 params
  attn.c_attn.weight                       (768, 2304)             1,769,472 params
  attn.c_proj.bias                         (768,)                        768 params
  attn.c_proj.weight                       (768, 768)                589,824 params
  ln_1.bias                                (768,)                        768 params
  ln_1.weight                              (768,)                        768 params
  ln

---

## GPT-2 Architecture Summary

Now let's see the complete architecture with all hyperparameters and weight tensor explanations.


In [29]:
def explain_architecture(tensors: dict, config: dict):
    """Explain what each weight tensor does."""
    
    print("\n" + "=" * 75)
    print("🧠 UNDERSTANDING THE ARCHITECTURE")
    print("=" * 75)
    
    # Get shapes for explanation
    wte = tensors["wte.weight"]
    wpe = tensors["wpe.weight"]
    
    n_vocab, n_embd = wte.shape
    n_ctx, _ = wpe.shape
    
    # Count layers
    n_layer = sum(1 for k in tensors if k.startswith("h.") and ".attn.c_attn.weight" in k)
    
    # Get number of attention heads from config
    n_head = config["n_head"]
    head_dim = n_embd // n_head
    
    print(f"""
    ┌─────────────────────────────────────────────────────────────────────┐
    │                        GPT-2 Architecture                           │
    ├─────────────────────────────────────────────────────────────────────┤
    │                                                                     │
    │  HYPERPARAMETERS:                                                   │
    │    • Vocabulary size (n_vocab):     {n_vocab:>6}                         │
    │    • Max sequence length (n_ctx):   {n_ctx:>6}                         │
    │    • Embedding dimension (n_embd):  {n_embd:>6}                         │
    │    • Number of layers (n_layer):    {n_layer:>6}                         │
    │    • Number of heads (n_head):      {n_head:>6}                         │
    │    • Head dimension:                {head_dim:>6}                         │
    │                                                                     │
    ├─────────────────────────────────────────────────────────────────────┤
    │                                                                     │
    │  WEIGHT TENSORS EXPLAINED:                                          │
    │                                                                     │
    │  Embeddings:                                                        │
    │    wte.weight [{n_vocab}, {n_embd}]                                     │
    │      └─ Token embeddings: each token ID → {n_embd}-dim vector         │
    │                                                                     │
    │    wpe.weight [{n_ctx}, {n_embd}]                                      │
    │      └─ Position embeddings: each position → {n_embd}-dim vector      │
    │                                                                     │
    │  Per Transformer Block (×{n_layer}):                                     │
    │    h.X.ln_1.weight/bias [{n_embd}]                                     │
    │      └─ LayerNorm before attention                                  │
    │                                                                     │
    │    h.X.attn.c_attn.weight [{n_embd}, {3*n_embd}]                          │
    │      └─ Combined Q, K, V projection (this is where Q,K,V come from!)│
    │         Projects input → [Q, K, V] concatenated                     │
    │                                                                     │
    │    h.X.attn.c_proj.weight [{n_embd}, {n_embd}]                            │
    │      └─ Output projection after attention                           │
    │                                                                     │
    │    h.X.ln_2.weight/bias [{n_embd}]                                     │
    │      └─ LayerNorm before MLP                                        │
    │                                                                     │
    │    h.X.mlp.c_fc.weight [{n_embd}, {4*n_embd}]                             │
    │      └─ MLP first layer (expand to 4× dimension)                    │
    │                                                                     │
    │    h.X.mlp.c_proj.weight [{4*n_embd}, {n_embd}]                           │
    │      └─ MLP second layer (project back to {n_embd})                   │
    │                                                                     │
    │  Final:                                                             │
    │    ln_f.weight/bias [{n_embd}]                                         │
    │      └─ Final LayerNorm before output                               │
    │                                                                     │
    │  Note: GPT-2 ties wte.weight for input AND output (no separate lm_head)│
    │                                                                     │
    └─────────────────────────────────────────────────────────────────────┘
    """)

# Explain the architecture
explain_architecture(tensors, config)



🧠 UNDERSTANDING THE ARCHITECTURE

    ┌─────────────────────────────────────────────────────────────────────┐
    │                        GPT-2 Architecture                           │
    ├─────────────────────────────────────────────────────────────────────┤
    │                                                                     │
    │  HYPERPARAMETERS:                                                   │
    │    • Vocabulary size (n_vocab):      50257                         │
    │    • Max sequence length (n_ctx):     1024                         │
    │    • Embedding dimension (n_embd):     768                         │
    │    • Number of layers (n_layer):        12                         │
    │    • Number of heads (n_head):          12                         │
    │    • Head dimension:                    64                         │
    │                                                                     │
    ├──────────────────────────────────────────────────────

---

## ✅ Step 1 Complete!

We've successfully:
1. Downloaded GPT-2 model weights from HuggingFace
2. Parsed the SafeTensors format manually (no library needed!)
3. Understood **tensors** as multi-dimensional arrays of learned parameters
4. Explored **token embeddings** - how words become vectors
5. Explored **position embeddings** - how the model knows word order
6. Learned about **attention heads** - how the model focuses on different patterns

### Next Step
We'll implement the **tokenizer** to convert text → tokens, so we can actually feed text into the model!
